In [1]:
from flask import Flask
from flask import Blueprint, request
from werkzeug.utils import secure_filename
import numpy as np
import tensorflow as tf
import pandas as pd
from pyspark.sql import SparkSession
from pyspark.ml.recommendation import ALS, ALSModel
from scipy.spatial.distance import cosine
import joblib
from tensorflow.keras.applications import MobileNetV2
import json
import uuid

# 필요한 PyTorch 라이브러리 불러오기
import torch
import torch.nn as nn
from torchvision import transforms
from torchvision.utils import save_image
from PIL import Image
from IPython.display import Image as display_image

# 해시태그 관련

In [2]:
img_shape = (160, 160, 3)

# 사전 학습된 MobileNetV2 모델을 base_model로 저장
base_model = MobileNetV2(input_shape=img_shape, include_top=False, weights='imagenet')
# ***dropout등 튜닝을 해도 될거같다!
# include_top: 네트워크의 최상단에 완전연결 레이어를 넣을지 여부
# 참고로 마지막 분류 계층 ("상단")은 피쳐 추출에 그리 유용하지 않으므로 include_top=False를 지정해 맨 위에 분류 계층을 제외
# weights: 'imagenet' (ImageNet에 대한 선행 학습)

# tf.keras.layers.GlobalAveragePooling2D 레이어를 사용하여 피처를 이미지 당 하나의 1280 요소 벡터로 변환
global_average_layer = tf.keras.layers.GlobalAveragePooling2D()

# base_model에 global_average_layer 쌓기
neural_network = tf.keras.Sequential([
  base_model,
  global_average_layer,
])
# joblib.dump(neural_network,'./model/mobilenetv2.pkl')

In [3]:
spark = SparkSession.builder.master('local').appName('all').getOrCreate()

In [4]:
#######모델 불러오기 (dnn feature뽑아오기,als 해시태그 추천)
# model_path='/content/drive/MyDrive/insta_crawling/Hashtag_Team16/modeling (1)/mobilenetv2.pkl'
# neural_network = joblib.load(model_path)
als_model = ALSModel.load('modeling/als')

#######데이터 불러오기 (pics 이미지의 피처 데이터프레임,hashtags_df 모든해시태그, recommender_df 추천df)
recommender_df=joblib.load('recommender_df_0424_1.pkl')
hashtags_df=joblib.load('hashtags_df_0424_1.pkl')

In [5]:
def prepare_image(img_path, height=160, width=160):
    # 신경망에 맞게 이미지를 다운 샘플링 및 스케일링
    img = tf.io.read_file(img_path) # 불러(읽어)오기
    #img = tf.image.decode_jpeg(img) # [height, width, num_channels]인 3차원 배열을 반환
    img = tf.image.decode_image(img)
    img = tf.cast(img, tf.float32) # 정수형으로 바꾼경우 소수점을 버린다 boolean일때는 True면 1, False면 0을 출력
    img = (img/127.5) - 1
    img = tf.image.resize(img, (height, width))
    # 컬러 이미지의 차원에 맞게 회색조 이미지 형태변경
    if img.shape != (160, 160, 3):
        img = tf.concat([img, img, img], axis=2)
    return img

def extract_features(image, neural_network):
    # input받은 이미지를 1280개의 deep feature들로 구성된 벡터로 반환
    image_np = image.numpy() # numpy형태로 변환
    images_np = np.expand_dims(image_np, axis=0) # 차원추가([]를 씌워준다)
    deep_features = neural_network.predict(images_np)[0]
    return deep_features

# 코사인 유사성에 기반한 K개의 최근접이웃을 찾는 함수 
def find_neighbor_vectors(image_path, k=5, recommender_df=recommender_df):
    # 비슷한 이미지에 대한 img_features(이미지 피쳐, 즉 사용자 벡터)를 찾는다.
    prep_image = prepare_image(image_path)
    pics = extract_features(prep_image, neural_network)
    rdf = recommender_df.copy()
    rdf['dist'] = rdf['deep_features'].apply(lambda x: cosine(x, pics))
    rdf = rdf.sort_values(by='dist')
    return rdf.head(k)
  

def generate_hashtags(image_path):
    fnv = find_neighbor_vectors(image_path, k=5, recommender_df=recommender_df)
    # 코사인 유사성에 기반하여 5개의 사용자 벡터의 평균을 구한다.
    features = []
    for item in fnv.features.values:
        features.append(item)

    avg_features = np.mean(np.asarray(features), axis=0)
    
    hashtag_features = als_model.itemFactors.toPandas()

    # 앞서 구한 이미지(사용자) 피쳐의 평균, 즉 avg_features을 hashtag_features와 dot product하여 새로운 dot_product열 생성
    hashtag_features['dot_product'] = hashtag_features['features'].apply(lambda x: np.asarray(x).dot(avg_features))
    
    # 가장 높은 dot product를 가진 해시태그 상위 10개 추출
    final_recs = hashtag_features.sort_values(by='dot_product', ascending=False).head(50)

    # hastag_id로 hashtag 찾아서 output에 저장
    output = []
    for hashtag_id in final_recs.id.values:
        output.append(hashtags_df.iloc[hashtag_id]['hashtag'])
    return output

# 이미지 스타일 전이 관련

In [6]:
# 인코더(Encoder) 정의
vgg = nn.Sequential(
    nn.Conv2d(3, 3, (1, 1)),
    nn.ReflectionPad2d((1, 1, 1, 1)),
    nn.Conv2d(3, 64, (3, 3)),
    nn.ReLU(), # relu1-1
    nn.ReflectionPad2d((1, 1, 1, 1)),
    nn.Conv2d(64, 64, (3, 3)),
    nn.ReLU(), # relu1-2
    nn.MaxPool2d((2, 2), (2, 2), (0, 0), ceil_mode=True),
    nn.ReflectionPad2d((1, 1, 1, 1)),
    nn.Conv2d(64, 128, (3, 3)),
    nn.ReLU(), # relu2-1
    nn.ReflectionPad2d((1, 1, 1, 1)),
    nn.Conv2d(128, 128, (3, 3)),
    nn.ReLU(), # relu2-2
    nn.MaxPool2d((2, 2), (2, 2), (0, 0), ceil_mode=True),
    nn.ReflectionPad2d((1, 1, 1, 1)),
    nn.Conv2d(128, 256, (3, 3)),
    nn.ReLU(), # relu3-1
    nn.ReflectionPad2d((1, 1, 1, 1)),
    nn.Conv2d(256, 256, (3, 3)),
    nn.ReLU(), # relu3-2
    nn.ReflectionPad2d((1, 1, 1, 1)),
    nn.Conv2d(256, 256, (3, 3)),
    nn.ReLU(), # relu3-3
    nn.ReflectionPad2d((1, 1, 1, 1)),
    nn.Conv2d(256, 256, (3, 3)),
    nn.ReLU(), # relu3-4
    nn.MaxPool2d((2, 2), (2, 2), (0, 0), ceil_mode=True),
    nn.ReflectionPad2d((1, 1, 1, 1)),
    nn.Conv2d(256, 512, (3, 3)),
    nn.ReLU(), # relu4-1, this is the last layer used
    nn.ReflectionPad2d((1, 1, 1, 1)),
    nn.Conv2d(512, 512, (3, 3)),
    nn.ReLU(), # relu4-2
    nn.ReflectionPad2d((1, 1, 1, 1)),
    nn.Conv2d(512, 512, (3, 3)),
    nn.ReLU(), # relu4-3
    nn.ReflectionPad2d((1, 1, 1, 1)),
    nn.Conv2d(512, 512, (3, 3)),
    nn.ReLU(), # relu4-4
    nn.MaxPool2d((2, 2), (2, 2), (0, 0), ceil_mode=True),
    nn.ReflectionPad2d((1, 1, 1, 1)),
    nn.Conv2d(512, 512, (3, 3)),
    nn.ReLU(), # relu5-1
    nn.ReflectionPad2d((1, 1, 1, 1)),
    nn.Conv2d(512, 512, (3, 3)),
    nn.ReLU(), # relu5-2
    nn.ReflectionPad2d((1, 1, 1, 1)),
    nn.Conv2d(512, 512, (3, 3)),
    nn.ReLU(), # relu5-3
    nn.ReflectionPad2d((1, 1, 1, 1)),
    nn.Conv2d(512, 512, (3, 3)),
    nn.ReLU() # relu5-4
)

In [7]:
# 디코더(Decoder) 정의
decoder = nn.Sequential(
    nn.ReflectionPad2d((1, 1, 1, 1)),
    nn.Conv2d(512, 256, (3, 3)),
    nn.ReLU(),
    nn.Upsample(scale_factor=2, mode='nearest'),
    nn.ReflectionPad2d((1, 1, 1, 1)),
    nn.Conv2d(256, 256, (3, 3)),
    nn.ReLU(),
    nn.ReflectionPad2d((1, 1, 1, 1)),
    nn.Conv2d(256, 256, (3, 3)),
    nn.ReLU(),
    nn.ReflectionPad2d((1, 1, 1, 1)),
    nn.Conv2d(256, 256, (3, 3)),
    nn.ReLU(),
    nn.ReflectionPad2d((1, 1, 1, 1)),
    nn.Conv2d(256, 128, (3, 3)),
    nn.ReLU(),
    nn.Upsample(scale_factor=2, mode='nearest'),
    nn.ReflectionPad2d((1, 1, 1, 1)),
    nn.Conv2d(128, 128, (3, 3)),
    nn.ReLU(),
    nn.ReflectionPad2d((1, 1, 1, 1)),
    nn.Conv2d(128, 64, (3, 3)),
    nn.ReLU(),
    nn.Upsample(scale_factor=2, mode='nearest'),
    nn.ReflectionPad2d((1, 1, 1, 1)),
    nn.Conv2d(64, 64, (3, 3)),
    nn.ReLU(),
    nn.ReflectionPad2d((1, 1, 1, 1)),
    nn.Conv2d(64, 3, (3, 3)),
)

In [8]:
decoder.eval()
vgg.eval()

vgg_path = './vgg_normalised.pth'
decoder_path = './decoder.pth'

decoder.load_state_dict(torch.load(decoder_path))
vgg.load_state_dict(torch.load(vgg_path))

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
vgg.to(device)
decoder.to(device)

vgg = nn.Sequential(*list(vgg.children())[:31]) # ReLU4_1까지만 사용하기 위해 뒤쪽은 자름.

In [9]:
# 피처맵에 대해 평균과 표준편차를 구해서 리턴하는 함수
def calc_mean_std(feat, eps=1e-5):
    size = feat.size()
    assert (len(size) == 4)
    N, C = size[:2]
    feat_var = feat.view(N, C, -1).var(dim=2) + eps #하나의 벡터로 만든 뒤, variance를 구해주고, 0이 되지 않도록 작은 값을 더해줌
    feat_std = feat_var.sqrt().view(N, C, 1, 1)
    feat_mean = feat.view(N, C, -1).mean(dim=2).view(N, C, 1, 1)
    return feat_mean, feat_std

In [10]:
def adaptive_instance_normalization(content_feat, style_feat):
    assert (content_feat.size()[:2] == style_feat.size()[:2])
    size = content_feat.size()
    style_mean, style_std = calc_mean_std(style_feat)
    content_mean, content_std = calc_mean_std(content_feat)

    # 평균(mean)과 표준편차(std)를 이용하여 정규화 수행
    normalized_feat = (content_feat - content_mean.expand(size)) / content_std.expand(size)
    # 정규화 이후에 style feature의 statistics를 가지도록 설정
    return normalized_feat * style_std.expand(size) + style_mean.expand(size)

In [11]:
def style_transfer(vgg, decoder, content, style, alpha=1.0):
    assert (0.0 <= alpha <= 1.0)
    content_f = vgg(content)
    style_f = vgg(style)
    feat = adaptive_instance_normalization(content_f, style_f)
    feat = feat * alpha + content_f * (1 - alpha)
    return decoder(feat)

In [12]:
def test_transform(size=512):
    transform_list = []
    if size != 0:
        transform_list.append(transforms.Resize(size))
    transform_list.append(transforms.ToTensor())
    transform = transforms.Compose(transform_list)
    return transform

content_tf = test_transform()
style_tf = test_transform()

In [13]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# 서버 실행

In [ ]:
from flask import Flask
from flask import Blueprint, request
from werkzeug.utils import secure_filename
import cv2
import base64

app = Flask(__name__)
# HTTP POST방식으로 전송된 이미지를 저장
@app.route('/', methods=['POST'])
def input_image():
    f = request.files['file']
    secure_name = secure_filename(f.filename)
    unique_name = str(uuid.uuid1())
    save_image_path = 'save_image/' + unique_name + secure_filename(f.filename)
    save_style_transfer_image_path = 'save_style_transfer_image/'+ unique_name + secure_filename(f.filename) + '.jpg'
    print(save_image_path)
    f.save(save_image_path)
    recommended_hashtags = generate_hashtags(save_image_path) #원래는 이미지 확장자 생각해서 적용해야함.
    
    content = content_tf(Image.open(save_image_path))
    style1 = style_tf(Image.open(str('E:/googledrive/aivle/big_proj/ML_API/style_image/style2_drawing/pic2.jfif')))

    style1 = style1.to(device).unsqueeze(0)
    content = content.to(device).unsqueeze(0)
    with torch.no_grad():
        output1 = style_transfer(vgg, decoder, content, style1, alpha=1.0)
    output1 = output1.cpu()

    save_image(output1, save_style_transfer_image_path)
    
    img = cv2.imread(save_style_transfer_image_path)
    jpg_img = cv2.imencode('.jpg', img)
    b64_string = base64.b64encode(jpg_img[1]).decode('utf-8')
    
    hashtags_dict = {'hashtags':recommended_hashtags,
                    'img': b64_string}
    hashtags_json = json.dumps(hashtags_dict, sort_keys=True)
    return hashtags_json

if __name__ == '__main__':
    app.run(host='0.0.0.0')

 * Serving Flask app '__main__' (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5000
 * Running on http://172.30.1.16:5000 (Press CTRL+C to quit)


save_image/43d6f3c6-c9b9-11ec-9e22-c03c596ba129jpg


C:\Users\User\anaconda3\envs\hashtaig\lib\site-packages\pyspark\sql\context.py:127: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  FutureWarning
C:\Users\User\anaconda3\envs\hashtaig\lib\site-packages\torch\nn\functional.py:780: UserWarning: Note that order of the arguments: ceil_mode and return_indices will changeto match the args list in nn.MaxPool2d in a future release.
  warnings.warn("Note that order of the arguments: ceil_mode and return_indices will change"
127.0.0.1 - - [02/May/2022 10:43:41] "POST / HTTP/1.1" 200 -


save_image/4f2e026e-c9bb-11ec-90ea-c03c596ba129jpg


127.0.0.1 - - [02/May/2022 10:58:14] "POST / HTTP/1.1" 200 -
